# Import Necessary Libraries

In [2]:
import gym    # "gym" refers to OpenAI Gym (https://gym.openai.com), which offers environments for our agents.
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical
from collections import deque
import matplotlib.pyplot as plt
import moviepy.editor as mpy

In [21]:
env = gym.make("Pong-v0")
obs = env.reset()
terminal = False
while not terminal:
    env.render()
    action = env.action_space.sample()
    obs, r, terminal, info = env.step(action) # take a random action
    # print(obs)
    # print(r)
    # print(info)

env.close()

TypeError: render() missing 1 required positional argument: 'mode'